In [1]:
import pandas as pd
import numpy as np
import os
import time
import glob
import matplotlib.pyplot as plt
from datetime import datetime
import math
%matplotlib inline

In [3]:
pd.set_option('max_columns', 30)
pd.options.display.float_format = '{:20,.2f}'.format

In [4]:
#kick = pd.DataFrame()

#for file in glob.glob("C:/Users/sastu/Downloads/Kickstarter_2018-10-18T03_20_48_880Z/*.csv"):
#    df = pd.read_csv(file)
#    kick = kick.append(df)
#    print("File appended: " + file)

In [5]:
#kick.to_csv("C:/Users/sastu/Downloads/Kick_data_appended.csv", index=False)

In [158]:
#k = pd.read_csv(r"C:\OSU\Sem3\BAN 5753\SAS Symposium\Kick_data_appended.csv", encoding='latin-1')

In [159]:
#k.columns

Index(['Unnamed: 0', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'creator',
       'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'launched_at', 'location', 'name', 'permissions', 'photo', 'pledged',
       'profile', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')

In [160]:
#k = k[['id', 'created_at','state_changed_at']]

In [162]:
#k.created_at = pd.to_datetime(k.created_at, unit='s')
#k.state_changed_at = pd.to_datetime(k.state_changed_at, unit='s')

In [164]:
#k['created_date'] = k.created_at.dt.date
#k['state_changed_date'] = k.state_changed_at.dt.date

In [166]:
#k = k.drop(['created_at', 'state_changed_at'], axis=1)

In [168]:
#k.columns = ['project_id', 'created_date', 'state_changed_date']

In [179]:
#kick = pd.merge(kick, k, on=['project_id'], how='inner')

In [180]:
#kick.drop_duplicates(inplace=True)

## Read the data from Kaggle and Webroots websites

### 1. Join the data on 'id' column
### 2. Keep useful columns from both the datasets (Dataset acquired from Kaggle is used as a primary dataset and some of the columns from the dataset acquired from Web Robots have added in the primary dataset)
### 3. Use 'static_usd_rate' column to convert 'usd_pledged' and 'goal_amount' columns
### 4. Keep only 'successful' and 'failed' projects based on 'project_state' column
### 5. Calculate 'campaign_length' column using 'launched' and 'deadline' columns

## Finally various features were engineered for my analysis

In [2]:
kick = pd.read_csv(r"C:\OSU\Sem3\BAN 5753\SAS Symposium\kickstarter_data_model.csv", encoding='latin-1')

In [5]:
kick.columns

Index(['project_id', 'project_name', 'project_blurb', 'main_category',
       'sub_category', 'project_state', 'country', 'deadline', 'launched',
       'campaign_length', 'backers_count', 'currency', 'static_usd_rate',
       'pledged_amount', 'pledged_amount_usd', 'goal_amount',
       'goal_amount_usd', 'staff_pick', 'spotlight', 'disable_communication'],
      dtype='object')

In [8]:
kick['launched'] = pd.to_datetime(kick.launched)
kick['deadline'] = pd.to_datetime(kick.deadline)

In [12]:
kick['launch_year'] = pd.DatetimeIndex(kick['launched']).year

In [13]:
kick['launch_month'] = pd.DatetimeIndex(kick['launched']).month

In [49]:
kick['launch_week'] = pd.DatetimeIndex(kick['launched']).week

In [14]:
kick['launch_day'] = kick['launched'].dt.day_name()

In [38]:
kick['launch_quarter'] = kick['launched'].dt.quarter

In [21]:
kick['campaign_less_than_1_month'] = np.where(kick.campaign_length<=31, 1, 0)

In [28]:
kick['goal_amount_less_than_75'] = np.where(kick.goal_amount_usd<=12500, 1, 0)

In [32]:
kick['project_blurb_length'] = kick.project_blurb.str.len()
kick['project_name_length'] = kick.project_name.str.len()
# presence of !
kick['project_name_exclaim'] = (kick.project_name.str[-1] == '!').astype(int)

# presence of ?
kick['project_name_question'] = (kick.project_name.str[-1] == '?').astype(int)

# number of words in the name
kick['project_name_words'] = kick.project_name.apply(lambda x: len(str(x).split(' ')))

# if name is uppercase
kick['project_name_is_upper'] = kick.project_name.str.isupper().astype(float)

In [113]:
kick['project_blurb_words'] = kick.project_blurb.apply(lambda x: len(str(x).split(' ')))

In [36]:
kick['goal_amount_category_perc'] =  kick.groupby(['main_category'])['goal_amount_usd'].transform(
                     lambda x: pd.qcut(x, [0, .35, .70, 1.0], labels =[1,2,3]))

In [37]:
kick['campaign_length_category_perc'] =  kick.groupby(['main_category'])['campaign_length'].transform(
                     lambda x: pd.qcut(x, [0, .35, .70, 1.0], labels =False, duplicates='drop'))

In [43]:
#creating a metric to see number of competitors for a given project in a given quarter
#number of participants in a given category, that launched in the same year and quarter and in the same goal bucket
ks_particpants_qtr = kick.groupby(['main_category','launch_year','launch_quarter','goal_amount_category_perc']).count()
ks_particpants_qtr=ks_particpants_qtr[['project_name']]
#since the above table has all group by columns created as index, converting them into columns
ks_particpants_qtr.reset_index(inplace=True)
ks_particpants_qtr.rename(index=str, columns={"project_name": "competitors_quarter"}, inplace=True)

In [47]:
#creating a metric to see number of competitors for a given project in a given month
#number of participants in a given category, that launched in the same year and month and in the same goal bucket
ks_particpants_mth=kick.groupby(['main_category','launch_year','launch_month','goal_amount_category_perc']).count()
ks_particpants_mth=ks_particpants_mth[['project_name']]
#since the above table has all group by columns created as index, converting them into columns
ks_particpants_mth.reset_index(inplace=True)
ks_particpants_mth.rename(index=str, columns={"project_name": "competitors_month"}, inplace=True)

,main_category,launch_year,launch_month,goal_amount_category_perc,competitors_month
0,Art,2009,4,1.00,1
1,Art,2009,5,1.00,5
2,Art,2009,5,2.00,5
3,Art,2009,6,1.00,2
4,Art,2009,6,2.00,2
5,Art,2009,6,3.00,1
6,Art,2009,7,1.00,5
7,Art,2009,7,2.00,4
8,Art,2009,8,1.00,2
9,Art,2009,8,2.00,7


In [52]:
#creating a metric to see number of competitors for a given project in a given week
#number of participants in a given category, that launched in the same year and week and in the same goal bucket
ks_particpants_wk=kick.groupby(['main_category','launch_year','launch_week','goal_amount_category_perc']).count()
ks_particpants_wk=ks_particpants_wk[['project_name']]
#since the above table has all group by columns created as index, converting them into columns
ks_particpants_wk.reset_index(inplace=True)
ks_particpants_wk.rename(index=str, columns={"project_name": "competitors_week"}, inplace=True)

,main_category,launch_year,launch_week,goal_amount_category_perc,competitors_week
0,Art,2009,17,1.00,1
1,Art,2009,18,2.00,1
2,Art,2009,19,1.00,1
3,Art,2009,19,2.00,1
4,Art,2009,20,1.00,3
5,Art,2009,20,2.00,2
6,Art,2009,21,1.00,1
7,Art,2009,22,2.00,1
8,Art,2009,23,2.00,1
9,Art,2009,23,3.00,1


In [65]:
kick = pd.merge(kick, ks_particpants_qtr, on=['main_category','launch_year','launch_quarter','goal_amount_category_perc'], 
                  how= 'inner')

In [66]:
kick = pd.merge(kick, ks_particpants_mth, on=['main_category','launch_year','launch_month','goal_amount_category_perc'], 
                  how= 'inner')

In [67]:
kick = pd.merge(kick, ks_particpants_wk, on=['main_category','launch_year','launch_week','goal_amount_category_perc'], 
                  how= 'inner')

In [75]:
#additional features from goal, pledge and backers columns
kick.loc[:,'goal_reached'] = kick['pledged_amount_usd'] / kick['goal_amount_usd'] # Pledged amount as a percentage of goal.
#The above field will be used to compute another metric
# In backers column, impute 0 with 1 to prevent undefined division.
kick.loc[kick['backers_count'] == 0, 'backers_count'] = 1 
kick.loc[:,'pledge_per_backer'] = kick['pledged_amount_usd'] / kick['backers_count'] # Pledged amount per backer.

In [76]:
#####################################################################
# using these metrics to create AVG_PPB and AVG_SUCCESS_RATE features
#####################################################################

#creating 2 metrics to get average pledge per backer for a category in a year according to the goal bucket it lies in and the success rate ie average pledged to goal ratio for the category and goal bucket in this year#creati 
#using pledge_per_backer and goal_reached (computed earlier) and averaging it by category in a launch year belonging to a goal bucket
ks_ppb_goal=pd.DataFrame(kick.groupby(['main_category','launch_year','goal_amount_category_perc'])['pledge_per_backer','goal_reached'].mean())
#since the above table has all group by columns created as index, converting them into columns
ks_ppb_goal.reset_index(inplace=True)
#renaming column
ks_ppb_goal.columns= ['main_category','launch_year','goal_amount_category_perc','avg_ppb_goal','avg_success_rate_goal']

#creating a metric: the success rate ie average pledged to goal ratio for the category in this year according to the duration buckets
ks_ppb_duration=pd.DataFrame(kick.groupby(['main_category','launch_year','campaign_length_category_perc'])['goal_reached'].mean())
#since the above table has all group by columns created as index, converting them into columns
ks_ppb_duration.reset_index(inplace=True)
#renaming column
ks_ppb_duration.columns= ['main_category','launch_year','campaign_length_category_perc','avg_success_rate_campaign_length']

In [80]:
kick = pd.merge(kick, ks_ppb_goal, on=['main_category','launch_year','goal_amount_category_perc'], how= 'inner')
kick = pd.merge(kick, ks_ppb_duration, on=['main_category','launch_year','campaign_length_category_perc'], how= 'inner')

In [134]:
kick.loc[:,'project_name_words_sd'] = np.sqrt((kick.project_name_words - 
                                           kick.loc[kick.staff_pick == True].groupby(['staff_pick'])['project_name_words'].mean()[1])**2)

In [135]:
kick.loc[:,'project_blurb_words_sd'] = np.sqrt((kick.project_blurb_words - 
                                           kick.loc[kick.staff_pick == True].groupby(['staff_pick'])['project_blurb_words'].mean()[1])**2)

In [183]:
kick.created_date = pd.to_datetime(kick.created_date)
kick.state_changed_date = pd.to_datetime(kick.state_changed_date)

In [190]:
kick['days_between_create_and_launch'] = (kick.launched - kick.created_date).dt.days

In [191]:
kick['days_between_launch_and_state_change'] = (kick.state_changed_date - kick.launched).dt.days

In [194]:
kick.to_csv("C:\OSU\Sem3\BAN 5753\SAS Symposium\kick_data_new.csv", index=False)

In [6]:
kick = pd.read_csv("C:\OSU\Sem3\BAN 5753\SAS Symposium\kick_data_new.csv")

In [7]:
kick.shape

(163452, 50)

In [8]:
kick.columns

Index(['project_id', 'project_name', 'project_blurb', 'main_category',
       'sub_category', 'project_state', 'country', 'deadline', 'launched',
       'campaign_length', 'backers_count', 'currency', 'static_usd_rate',
       'pledged_amount', 'pledged_amount_usd', 'goal_amount',
       'goal_amount_usd', 'staff_pick', 'spotlight', 'disable_communication',
       'launch_year', 'launch_month', 'launch_day',
       'campaign_less_than_1_month', 'goal_amount_less_than_75',
       'project_blurb_length', 'project_name_length', 'project_name_exclaim',
       'project_name_question', 'project_name_words', 'project_name_is_upper',
       'goal_amount_category_perc', 'campaign_length_category_perc',
       'launch_quarter', 'launch_week', 'competitors_quarter',
       'competitors_month', 'competitors_week', 'goal_reached',
       'pledge_per_backer', 'avg_ppb_goal', 'avg_success_rate_goal',
       'avg_success_rate_campaign_length', 'project_blurb_words',
       'project_name_words_sd', 'pr